# Task: Predict Health Insurance Owners' who will be interested in Vehicle Insurance

An insurance company has provided Health Insurance to its customers now they want a model to predict whether the policyholders (customers) from past year will also be interested in Vehicle Insurance provided by the company.

Before building model, let's explore the dataset and get some insights from data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
%matplotlib inline

train=pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
#test=pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')

print("First five rows of training dataset are:")
train.head()

Information about columns:

* id  -	Unique ID for the customer
* Gender  -	Gender of the customer
* Age  - 	Age of the customer
* Driving_License  - 	0 : Customer does not have DL, 1 : Customer already has DL
* Region_Code  -	Unique code for the region of the customer
* Previously_Insured  -	1 : Customer already has Vehicle Insurance, 0 : Customer doesn't have Vehicle Insurance
* Vehicle_Age  -	Age of the Vehicle
* Vehicle_Damage  -  1 : Customer got his/her vehicle damaged in the past. 0 : Customer didn't get his/her vehicle damaged in the past.
* Annual_Premium  -	The amount customer needs to pay as premium in the year
* PolicySalesChannel  -	Anonymized Code for the channel of outreaching to the customer ie.,Different Agents, Over Mail, Over Phone, In Person, etc.
* Vintage  -	Number of Days, Customer has been associated with the company
* Response  -	1 : Customer is interested, 0 : Customer is not interested


We are given two csv files in this dataset: train and test. For EDA, I will use only train set.

Let's have a look at data types of different columns.

In [ ]:
train.info()

In [ ]:
print(f"Train set has {train.shape[0]} rows and {train.shape[1]} columns.")
print(f"Missing values are present in data: {train.isnull().sum().any()}")

# EDA

Features Response, Driving_License and Previously_Insured are alreadyencoded in data. For EDA purpose, I am converting these into object type for easy understanding in visualizations.

In [ ]:
train_eda = train.copy()
cols=['Driving_License','Previously_Insured','Response']
for col in cols:
    train_eda[col] = train_eda[col].map({0:'No',1:'Yes'})

## Target variable: Response

In [ ]:
sns.countplot(train_eda['Response'],palette='rocket')
plt.title("Target variable Distribution in data");

* So, our data is highly imbalanced.

> I made two notebooks showing 2 ways to handle imbalanced data. Give them a look:
    [down-sampling-majority-class-6-classification-algo](https://www.kaggle.com/ayushikaushik/down-sampling-majority-class-6-classification-algo)
    and
    [up-sampling-to-tackle-unbalanced-dataset](https://www.kaggle.com/ayushikaushik/up-sampling-to-tackle-unbalanced-dataset)

> [Here](https://www.analyticsvidhya.com/blog/2017/03/imbalanced-data-classification/) is a good article to understand problems caused by imbalanced article and ways to handle them.

## Gender

In [ ]:
sns.countplot(train_eda['Gender'],palette='summer')
plt.title("Gender Distribution in data");

* There are more samples of gender male.
* Seems as if males are more concerned about health risks.  ;)

## Age

In [ ]:
print("Age distribution according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Age",shade = True)
facetgrid.set(xlim = (0,train_eda["Age"].max()))
facetgrid.add_legend();

* Young people below 30 are not interested in vehicle insurance. Reasons could be lack of experience, less maturity level and they don't have expensive vehicles yet.
* People aged between 30-60 are more likely to be interested.

## Driving License

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Driving_License'])

> ***You should always have driving license while driving***

* Here also majority observations have a driving license.


## Region-wise Distribution

In [ ]:
train_eda['Region_Code'].value_counts().plot(kind='barh',cmap='Accent',figsize=(12,10));

* Meaning of these codes has not been provided.
* Most of the data is collected from people living in region with code 28

## Previously-insured

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Previously_Insured'])

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Previously_Insured']).plot(kind='bar');

* Those who already have an insurance are not interested. This was obvious man!


## Vehicle Age

In [ ]:
plt.rcParams['figure.figsize']=(6,8)
color = ['yellowgreen','gold',"lightskyblue"]
train_eda['Vehicle_Age'].value_counts().plot.pie(y="Vehicle_Age",colors=color,explode=(0.02,0,0.3),startangle=50,shadow=True,autopct="%0.1f%%")
plt.axis('on');

In [ ]:
sns.countplot(train_eda['Vehicle_Age'],hue=train_eda['Response'],palette='autumn');

* More than half of the data (52%) has samples with vehicle age between 1-2 years.
* We can't say from second graph that people with vehicle age between 1-2 years are more interested because other category '>2 years' has very few observations.


## Vehicle Damage

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Vehicle_Damage']).plot(kind='bar');

*  Customers who got his/her vehicle damaged in the past is more likely to be interested in insurance. May be because he has first-hand experience of its pros and cons.
* Ah! I want a version of 'Prevention is better than cure' for this situation.

## Annual Premium

In [ ]:
print("Annual Premium distribution according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Annual_Premium",shade = True)
facetgrid.set(xlim = (0,train_eda["Annual_Premium"].max()))
facetgrid.add_legend();

* I don't think it gives much knowledge.
* Outliers can be present in this feature.

## PolicySalesChannel

Anonymized Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc.

In [ ]:
print("Policy_Sales_Channel distribution according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Policy_Sales_Channel",shade = True)
facetgrid.set(xlim = (0,train_eda["Policy_Sales_Channel"].max()))
facetgrid.add_legend();

* This graph seems interesting. But to clearly extract insights we need meaning of these codes.

## Vintage

Number of Days, Customer has been associated with the company

In [ ]:
print("Vintage feature according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Vintage",shade = True)
facetgrid.set(xlim = (0,train_eda["Vintage"].max()))
facetgrid.add_legend();

* Our target variable is not much affected by this feature. It can be dropped.

In [ ]:
print("Correlation matrix-")
plt.rcParams['figure.figsize']=(8,6)
sns.heatmap(train.corr(),cmap='Spectral');

In [ ]:
train.corr()[:-1]['Response'].sort_values().round(2)

I will remove least correlated features for modelling.

# Creating classification Model

In [ ]:
#creating a checkpoint
df4model = train.copy()
#dropping Vintage column as suggested by EDA
df4model.drop(['id','Vintage'],axis=1,inplace=True)
#checking target variable
df4model.Response.value_counts()

To make this data balanced let's upsample the minority class using sklearn library resample.

To prevent any data leakage I will first split into train and test subsets and then do upsampling as suggested by [@antaresnyc](https://www.kaggle.com/antaresnyc).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df4model.drop(['Response'], axis = 1), 
                                                    df4model['Response'], test_size = 0.2)

In [ ]:
print(f"Target variable disribution in train set: \n{y_train.value_counts()}\n\nand in test set: \n{y_test.value_counts()}")

## Up-sampling

In [ ]:
#combining train features and target
df = pd.concat([X_train,y_train],axis=1)

from sklearn.utils import resample,shuffle
df_majority = df[df['Response']==0]
df_minority = df[df['Response']==1]
df_minority_upsampled = resample(df_minority,replace=True,n_samples=y_train.value_counts()[0],random_state = 123)
balanced_df = pd.concat([df_minority_upsampled,df_majority])
balanced_df = shuffle(balanced_df)
balanced_df.Response.value_counts()

Now, I will convert categorical columns into numerical ones.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder= OrdinalEncoder()
cat_cols=['Gender','Vehicle_Damage']
balanced_df[cat_cols] = encoder.fit_transform(balanced_df[cat_cols])
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

dummy = pd.get_dummies(balanced_df['Vehicle_Age'],drop_first=True)
features = pd.concat([dummy,balanced_df],axis=1)
features.drop('Vehicle_Age',axis=1,inplace=True)

features.head()

In [ ]:
#to get uniform output
features = features.astype('float64')
X_train = features.drop('Response',axis=1)
y_train = features['Response']

#creating dummies in test set
dummy1 = pd.get_dummies(X_test['Vehicle_Age'],drop_first=True)
X_test = pd.concat([dummy1,X_test],axis=1)
X_test.drop('Vehicle_Age',axis=1,inplace=True)

# Logistic Regression

In [ ]:
logisticRegression = LogisticRegression(max_iter = 10000)
logisticRegression.fit(X_train, y_train)
predictions = logisticRegression.predict(X_test)
print(f"Accuracy score is {100*accuracy_score(y_test,predictions).round(2)}\nROC-AUC score is {100*roc_auc_score(y_test,predictions).round(2)}")

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print(f"Accuracy score is {100*accuracy_score(y_test,rfc_pred).round(2)}\nROC-AUC score is {100*roc_auc_score(y_test,rfc_pred).round(2)}")

In [ ]:
rfc_preds = rfc.predict_proba(X_test)
print("AUC score after taking probabilities predictions and not classes predictions is")
roc_auc_score(y_test, rfc_preds[:,1], average = 'weighted')

In [ ]:
X_train.columns= ['less than 1 Year','greater than 2 Years', 'Gender', 'Age','Driving_License',
                  'Region_Code', 'Previously_Insured', 'Vehicle_Damage', 'Annual_Premium','Policy_Sales_Channel']
X_test.columns= ['less than 1 Year','greater than 2 Years', 'Gender', 'Age','Driving_License',
                  'Region_Code', 'Previously_Insured', 'Vehicle_Damage', 'Annual_Premium','Policy_Sales_Channel']

from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
print(f"Accuracy score is {100*accuracy_score(y_test,xgb_pred).round(2)}\nROC-AUC score is {100*roc_auc_score(y_test,xgb_pred).round(2)}")

In [ ]:
xgb_preds = xgb.predict_proba(X_test)
roc_auc_score(y_test, xgb_preds[:,1], average = 'weighted')